# Harkatu.am scraping, analysis of top taxpayers in Armenia

In [1]:
import requests
from scrapy.http import TextResponse
import pandas as pd
from selenium import webdriver
import time

In [2]:
url='https://www.harkatu.am/companies/hy?0#p/1'
browser=webdriver.Chrome()
browser.get(url)

In [3]:
all_urls=['https://www.harkatu.am/companies/hy?0#p/{}'.format(i) for i in range(1,9)] #all urls from which info to be scraped

In [4]:
all_urls

['https://www.harkatu.am/companies/hy?0#p/1',
 'https://www.harkatu.am/companies/hy?0#p/2',
 'https://www.harkatu.am/companies/hy?0#p/3',
 'https://www.harkatu.am/companies/hy?0#p/4',
 'https://www.harkatu.am/companies/hy?0#p/5',
 'https://www.harkatu.am/companies/hy?0#p/6',
 'https://www.harkatu.am/companies/hy?0#p/7',
 'https://www.harkatu.am/companies/hy?0#p/8']

In [5]:
page=browser.page_source
response=TextResponse(url=url, body=page, encoding = 'utf-8')

In [6]:
response.css('td.sales-volume>div.volume::text').extract()[1::2][:25] #taxes

['13,570,605',
 '12,830,020',
 '6,684,445',
 '4,544,427',
 '4,295,157',
 '3,955,956',
 '3,769,765',
 '3,753,700',
 '3,414,762',
 '2,794,111',
 '2,590,033',
 '2,389,817',
 '2,356,659',
 '2,061,793',
 '1,751,630',
 '1,564,916',
 '1,485,612',
 '1,403,194',
 '1,399,955',
 '1,363,849',
 '1,316,943',
 '1,279,783',
 '1,276,919',
 '1,262,660',
 '1,261,087']

In [7]:
response.css('h1> a> span::text').extract()

['ԳԱԶՊՐՈՄ ԱՐՄԵՆԻԱ',
 'ԳՐԱՆԴ ՏՈԲԱԿՈ',
 'ԶԱՆԳԵԶՈՒՐԻ ՊՂՆՁԱՄՈԼԻԲԴԵՆԱՅԻՆ ԿՈՄԲԻՆԱՏ',
 'ՀԱՅԱՍՏԱՆԻ ԷԼԵԿՏՐԱԿԱՆ ՑԱՆՑԵՐ',
 'ԳԵՈՊՐՈՄԱՅՆԻՆԳ ԳՈԼԴ',
 'ԻՆՏԵՐՆԵՅՇՆԼ ՄԱՍԻՍ ՏԱԲԱԿ',
 'ՍԻՓԻԷՍ ՕԻԼ ՔՈՐՓՈՐԵՅՇՆ',
 'ՖԼԵՇ',
 'ԱԿՎԻԼՈՆ ԹՐԵՅԴ',
 'ԱՐՄԵՆԻԱ ՄԻՋԱԶԳԱՅԻՆ ՕԴԱՆԱՎԱԿԱՅԱՆՆԵՐ',
 'ՄԱՔՍ ՊԵՏՐՈԼ',
 'ՀԱՅԿԱԿԱՆ ԱՏՈՄԱՅԻՆ ԷԼԵԿՏՐԱԿԱՅԱՆ',
 'ԱՐՄԵՆԻԱՆ ՄԱՅՆԻՆԳ ՔՈՆԹՐԱՔԹՈՐ',
 'ՅՈՒՔՈՄ',
 'ԳԱԶՊՐՈՄ ԱՐՄԵՆԻԱ ՓԲԸ ՏՐԱՆՍԳԱԶ',
 'ՍԻԹԻ',
 'ԳՐԱՆԴ ՄԱՍՏԵՐ',
 'ԳՐԱՆԴ ՔԵՆԴԻ',
 'ՄՈԲԱՅԼ ՍԵՆԹՐ ԱՐԹ',
 'ԱՄԵՐԻԱԲԱՆԿ',
 'ԱԼԵՔՍ ԷՆԴ ՀՈԼԴԻՆԳ',
 'ՎԵՈԼԻԱ ՋՈՒՐ',
 'ՀԱՅԲԻԶՆԵՍԲԱՆԿ ՓԲԸ',
 'ԱՎՏՈՆԵՐԿՐՈՂ -10',
 'ԵՐԵՎԱՆԻ ՋԵՐՄԱԷԼԵԿՏՐԱԿԵՆՏՐՈՆ']

In [8]:
response.css('div.industry::text').extract()

['Գազի մատակարարում',
 'Ծխախոտային արտադրատեսակներ',
 'Մետաղական հանքարդյունաբերություն',
 'Էլեկտրականության մատակարարում',
 'Մետաղական հանքարդյունաբերություն',
 'Ծխախոտային արտադրատեսակներ',
 'Վառելիքի վաճառք, Վառելիքի ներմուծում',
 'Մանրածախ առևտուր, Մեծածախ առևտուր',
 'Հագուստի վաճառք, Մեծածախ առևտուր',
 'Տրանսպորտ',
 'Վառելիքի ներմուծում, Վառելիքի վաճառք',
 'Էլեկտրականության արտադրություն',
 'Տրանսպորտ, Ցամաքային տրանսպորտ',
 'Հեռահաղորդակցություն',
 'Գազի մատակարարում',
 'Արդյունաբերություն',
 'Սննդամթերքի արտադրություն',
 'Ներմուծում, Առևտուր',
 'Բանկային ոլորտ',
 'Ջրամատակարարում և կոյուղի',
 'Բանկային ոլորտ',
 'Էլեկտրականության արտադրություն']

In [9]:
### SCRAPING ALL THE NECESSARY INFORMATION FROM ALL PAGES USING FOR LOOP ###
all_taxes=[]
all_companies=[]
all_industries=[]
for i in all_urls:
    browser.get(i)
    time.sleep(3) #giving some more time for the new page to load and info to be scraped
    page=browser.page_source
    response=TextResponse(url=i, body=page, encoding = 'utf-8')
    taxes=response.css('td.sales-volume>div.volume::text').extract()[1::2][:25] # all taxes
    companies=response.css('h1> a> span::text').extract() # all companies
    industries=response.css('div.industry::text').extract() # all industries
    all_taxes.extend(taxes)
    all_companies.extend(companies)
    all_industries.extend(industries)

In [10]:
print(len(all_taxes))
print(len(all_companies))
print(len(all_industries)) #some companies do not have industry indication, that is why the count is not the same

200
200
183


In [12]:
all_taxes=[i.replace(",","") for i in all_taxes] #eliminating commas to convert to integers later
all_taxes=pd.Series(all_taxes).astype(int)

In [13]:
all_taxes.mean() #the average taxes paid per annum

809770.225

In [14]:
all_industries=pd.Series(all_industries)
all_industries.value_counts()[:5]

Բանկային ոլորտ             14
Խմիչքների արտադրություն    11
Արդյունաբերություն          6
Ներմուծում                  6
Ծրագրային ապահովում         5
dtype: int64

The top industries that pay taxes are banking, drinks production and manufacturing.